# Algorithms for Twitter Bot Analysis

In [159]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.mlab import PCA
from sklearn import tree
from sklearn.metrics import roc_curve, auc

### Reading the data from The CSV files

In [146]:
twitterBots = pd.read_csv('../data/bots_data.csv',encoding='latin-1')
twitterNonBots=pd.read_csv('../data/nonbots_data.csv',encoding='latin-1')
user=pd.read_csv('../data/training_data_2_csv_UTF.csv',encoding='latin-1')
test=pd.read_csv('../data/test_data_4_students.csv')


### Cleaning Training Data and converting it to boolean & numeric values

In [147]:
df=user
df['favourites_count'].replace(['None'], '0')
df['default_profile'].replace(['None'], False)
df['default_profile_image'].replace(['None'], False)
df['screen_name']=df['screen_name'].str.contains('bot', na=False)
df['name']=df['name'].str.contains('bot', na=False)
df['description']=df['description'].str.contains('bot', na=False)
df['created_at'] = df['created_at'].str.replace('"', '')

df[:5]

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,8.160000e+17,"""815745789754417152""",False,"""Houston, TX""",False,"""https://t.co/dnWuDbFRkt""",1291,0,10,Mon Jan 02 02:25:26 +0000 2017,0,False,78554,"""en""","{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",True,False,False,False,1
1,4.843621e+09,4843621225,False,"Templeville town, MD, USA",False,NaN,1,349,0,2/1/2016 7:37,38,False,31,en,null,True,False,False,False,1
2,4.303727e+09,4303727112,True,NaN,True,https://t.co/P1e1o0m4KC,1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,False,713,en,"{'retweeted': False, 'is_quote_status': False,...",True,False,False,False,1
3,3.063139e+09,3063139353,False,NaN,False,NaN,33,0,8,2/25/2015 20:11,0,False,676,en,Construction of human anti-tetanus single-chai...,True,True,False,False,1
4,2.955142e+09,2955142070,False,"Dublin, United States",False,NaN,11,745,0,1/1/2015 17:44,146,False,185,en,null,False,False,False,False,1


###  Calculating the status per day from the day account was created

In [5]:
df['created_at']=pd.to_datetime(df['created_at'])
import datetime
now = datetime.datetime.now()
df['today']=now
df['daysTillNow'] = df['today']-df['created_at']
df['daysTillNow']=df['daysTillNow'].dt.days
df['statusPerDay']=df['statuses_count']/df['daysTillNow']

### Cleaning Test data and doing the same work we did for Training data

In [7]:
tf=test
tf = tf.rename(columns={'favourites_count': 'favourites_count'})

tf['favorites_count']=tf['favorites_count'].replace(['None'], 0)
tf['default_profile']=tf['default_profile'].replace(['None'], False)
tf['default_profile_image']=tf['default_profile_image'].replace(['None'], False)
tf['favorites_count']=tf['favorites_count'].replace(['FALSE'], False)
tf['favorites_count']=tf['favorites_count'].replace(['TRUE'], True)
tf['favorites_count']=tf['favorites_count'].replace(['None'], False)
tf['default_profile']=tf['default_profile'].replace(['FALSE'], False)
tf['default_profile']=tf['default_profile'].replace(['TRUE'], True)
tf['default_profile']=tf['default_profile'].replace(['None'], False)
tf['default_profile_image']=tf['default_profile_image'].replace(['FALSE'], False)
tf['default_profile_image']=tf['default_profile_image'].replace(['TRUE'], True)
tf['default_profile_image']=tf['default_profile_image'].replace(['None'], False)
tf['verified']=tf['verified'].replace(['TRUE'], True)
tf['verified']=tf['verified'].replace(['FALSE'], False)
tf['verified']=tf['verified'].replace(['None'], False)
tf['followers_count']=tf['followers_count'].replace(['None'], 0)
tf['friends_count']=tf['friends_count'].replace(['None'], 0)
tf['statuses_count']=tf['statuses_count'].replace(['None'], 0)
tf['screen_name']=tf['screen_name'].str.contains('bot', na=False)
tf['name']=tf['name'].str.contains('bot', na=False)
tf['description']=tf['description'].str.contains('bot', na=False)
tf['created_at'] = tf['created_at'].str.replace('"', '')
tf['created_at']=pd.to_datetime(tf['created_at'])
tf['today']=now
tf['daysTillNow'] = tf['today']-tf['created_at']
tf['daysTillNow']=tf['daysTillNow'].dt.days
tf['statusPerDay']=tf['statuses_count']/tf['daysTillNow']

### Splitting the data

In [32]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df[['followers_count','screen_name',
                                                        'friends_count','description','verified','statusPerDay',
                                                        'favourites_count','default_profile','default_profile_image','name']],df['bot'] ,test_size=0.3, random_state=0)

In [25]:
testData=tf[['followers_count','friends_count','description',
             'verified',
             'favorites_count','statusPerDay','default_profile_image','default_profile','name']]
testData=testData[:575]


### Analysing which feature will help in our prediction

In [161]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)
print ("Features sorted by their score:")
print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_),X_train.columns.values.tolist()), 
             reverse=True))

Features sorted by their score:
[(0.41810000000000003, 'friends_count'), (0.221, 'followers_count'), (0.14680000000000001, 'statusPerDay'), (0.093899999999999997, 'verified'), (0.074300000000000005, 'favourites_count'), (0.023300000000000001, 'description'), (0.0088000000000000005, 'default_profile_image'), (0.0076, 'default_profile'), (0.0054000000000000003, 'screen_name'), (0.00069999999999999999, 'name')]


### Fitting the training data  and predicting test data contained in training data and the test data from Kaggle using Random Forest

In [27]:
clf=RandomForestClassifier(n_estimators=175,max_features=3,min_samples_leaf=10)
model= clf.fit(X_train,Y_train)
predicted=model.predict(X_test)
predictedTest=model.predict(testData)
print('Accuracy of RandomForest is',accuracy_score(Y_test,predicted)*100,'%')
print(metrics.classification_report(Y_test,predicted))
print('AUC Score for Random forest is',metrics.roc_auc_score(Y_test, predicted))

Accuracy of RandomForest is 89.6428571429 %
             precision    recall  f1-score   support

          0       0.90      0.91      0.90       435
          1       0.90      0.89      0.89       405

avg / total       0.90      0.90      0.90       840

AUC Score for Random forest is 0.896083439762


### Using Decision tree to fit the model

In [28]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, Y_train)
predictedDT=dt.predict(X_test)
print('Accuracy of Decision Tree',accuracy_score(Y_test,predictedDT)*100,'%')
print(metrics.classification_report(Y_test,predictedDT))

Accuracy of Decision Tree 86.1904761905 %
             precision    recall  f1-score   support

          0       0.88      0.85      0.86       435
          1       0.84      0.87      0.86       405

avg / total       0.86      0.86      0.86       840



### Comparing Models

##### Clearly Random forest give us a higher accuracy

### Prediction For Kaggle test data and turning it in to excel file

In [29]:
output=test[:575]
output['bot']=predictedTest
predictedTest

C:\Users\saura\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


array([0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1,

In [149]:
nf=output[['id','bot']]
nf.to_csv('test.csv', index = False)